In [1]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
### General ###
import os
import copy
import tqdm
import pickle
import random
import warnings
warnings.filterwarnings("ignore")
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

### Data Wrangling ###
import numpy as np
import pandas as pd
from scipy import stats

### Machine Learning ###
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Deep Learning ###
import torch
from torch import nn
import torch.optim as optim
from torch.nn import functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Tabnet 
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor
from pickle import load,dump

from torch.nn import BCEWithLogitsLoss
### Make prettier the prints ###
from colorama import Fore
c_ = Fore.CYAN
m_ = Fore.MAGENTA
r_ = Fore.RED
b_ = Fore.BLUE
y_ = Fore.YELLOW
g_ = Fore.GREEN

## Read Data

In [3]:
train_features = pd.read_csv('train_features.csv')
train_targets_scored = pd.read_csv('train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('train_targets_nonscored.csv')

test_features = pd.read_csv('test_features.csv')
df = pd.read_csv('sample_submission.csv')

### We used nonscored MoA's to further train our model. However, 208 of them had less than 10 positive labels, so we removed them

In [4]:
# if a column in nonscored has less than 10 ones, drop it
count = 0
small = []
for col in range(len(train_targets_nonscored.columns)):
    if col==0:
        continue
    if sum(train_targets_nonscored.iloc[:, col])<10:
        small.append(col)
        count+=1
            
print(count)
train_targets_nonscored = train_targets_nonscored.drop(train_targets_nonscored.columns[small], axis = 1).reset_index(drop = True)
# 208 columns were dropped

208


In [5]:
# separate genes and cells columns
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [6]:
# seed everything
seed = 42

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seed(seed)

## cluster gene columns to get the (gene) pseudo labels



In [7]:
from sklearn.cluster import KMeans
def fe_cluster_genes(train, test, n_clusters_g = 20, SEED = 42):
    
    features_g = GENES
    
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans_genes = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        dump(kmeans_genes, open('kmeans_genes.pkl', 'wb'))
        train[f'clusters_{kind}'] = kmeans_genes.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_genes.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
    return train, test

train_features ,test_features = fe_cluster_genes(train_features, test_features)

## cluster cell columns to get the (cell) pseudo labels

In [8]:
def fe_cluster_cells(train, test, n_clusters_c = 5, SEED = 42):
    
    features_c = CELLS
    
    def create_cluster(train, test, features, kind = 'c', n_clusters = n_clusters_c):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans_cells = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        dump(kmeans_cells, open('kmeans_cells.pkl', 'wb'))
        train[f'clusters_{kind}'] = kmeans_cells.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_cells.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
    train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features, test_features = fe_cluster_cells(train_features, test_features)

## Drop control rows from the train set since these samples did not receive any drugs

In [9]:
train = train_features.merge(train_targets_nonscored, on='sig_id')
train = train.merge(train_targets_scored, on='sig_id')

ctrl = train[train['cp_type']=='ctl_vehicle'].reset_index(drop=True)
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)


test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)



In [10]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

## Augment data: we used the columns from the control data for augmentation

In [11]:
c = 1
num_samples = int(len(train)*np.floor(c))
sample_train = train.sample(num_samples, replace = False).reset_index(drop = True)
sample_ctrl = ctrl.sample(num_samples, replace = True).reset_index(drop = True)

sample_train[GENES+CELLS] += sample_ctrl[GENES+CELLS]
train = pd.concat([train, sample_train], axis = 0, ignore_index=True).reset_index(drop=True)


target_step1 = train[list(train_targets_nonscored.columns) +list(train_targets_scored.columns)]

target_step1=target_step1.drop('sig_id', axis = 1)
# 400 columns will be in target_step 1 where the first part is related to the nonscored columns
target_step2 = train[train_targets_scored.columns]
target_step2=target_step2.drop('sig_id', axis = 1)
#target_step_2 has 206 columns


In [12]:
# dose can be D1 or D2. cp_time can be 24, 48, or 72
train = pd.get_dummies(train, columns=['cp_time','cp_dose'])
test_ = pd.get_dummies(test, columns=['cp_time','cp_dose'])

### We trained the model in 2 steps. First to predict non-scored MoA's for the test set then to use test features + test non-scored MoA's for predicting the scored MoA's


In [13]:
feature_cols_step1 = [c for c in train.columns if c in train_features.columns and c not in ['sig_id']]
feature_cols_step2 = [c for c in train.columns if (c in train_features.columns or c in train_targets_nonscored.columns)and c not in ['sig_id']]


In [14]:
train_step1 = train[feature_cols_step1]
train_step2 = train[feature_cols_step2]
test = test_[feature_cols_step1]
X_test = test.values

In [15]:
# The score in Kaggle is calculated based on the LogitsLogLoss
class LogitsLogLoss(Metric):

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1 - y_true) * np.log(1 - logits + 5e-5) + y_true * np.log(logits + 5e-5)
        return np.mean(-aux)

### Best TabNet hyperparameters found by tuning 

In [16]:
MAX_EPOCH = 100

best_tabnet_params = dict(
    n_d = 32,
    n_a = 32,
    n_steps = 3,
    gamma = 2,
    lambda_sparse = 0.000001,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_params = dict(mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = seed,
    verbose = 10
)

## step 1 training

In [17]:
X_train, y_train = train_step1.values, target_step1.values
model = TabNetRegressor(**best_tabnet_params)
model.fit(
        X_train = X_train,
        y_train = y_train,
        max_epochs = MAX_EPOCH,
        patience = 20,
        batch_size = 1024, 
        virtual_batch_size = 32,
        num_workers = 1,
        drop_last = False,
        loss_fn = BCEWithLogitsLoss())

Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.14261 |  0:00:08s
epoch 10 | loss: 0.01425 |  0:01:33s
epoch 20 | loss: 0.01362 |  0:02:57s
epoch 30 | loss: 0.0131  |  0:04:23s
epoch 40 | loss: 0.01262 |  0:05:48s
epoch 50 | loss: 0.01239 |  0:07:14s
epoch 60 | loss: 0.01225 |  0:08:40s
epoch 70 | loss: 0.01216 |  0:10:05s
epoch 80 | loss: 0.01219 |  0:11:30s
epoch 90 | loss: 0.01216 |  0:12:55s


## get prediction (nonscored MoA for test data) for step 1


In [23]:
preds_step1 = model.predict(X_test)
preds_step1 = 1 / (1 + np.exp(-preds_step1))
preds_step1.shape
preds_step1 = preds_step1[:, 0:len(train_targets_nonscored.columns)-1]

In [24]:
preds_step1.shape

(3624, 194)

In [29]:
#building test features for step 2 (test features+ predicted non-scored MoA's) 
X_test_step2 = np.concatenate((X_test, preds_step1), axis = 1 )

In [30]:
X_test_step2.shape

(3624, 1091)

## the next 2 chunks are related to hyper parameter tuning and are commented since they take long

In [57]:
# building the search grid for tuning hyperparameters
# from sklearn.model_selection import ParameterGrid
# tabnet_params_2 = dict(
#     n_d = [32,64],
#     n_steps = [1, 3,5, 9],
#     gamma = [1, 1.5, 2],
#     lambda_sparse = [0.000001, 0.1],
#     optimizer_fn = [optim.Adam],
#     optimizer_params = [dict(lr = 0.02, weight_decay = 0.00001)],
#     mask_type = ["entmax"],
#     scheduler_params =[dict(mode = "min", patience = 20, min_lr = 1e-5, factor = 0.9)],
#     scheduler_fn = [ReduceLROnPlateau],
#     verbose = [10])

# len(ParameterGrid(tabnet_params_2))

36

In [ ]:
# # step 2 hyper parameter tuning
# with open('checkpoints.txt', 'w') as chck:
#     scores_auc_all = []
#     test_cv_preds = []
#     MAX_EPOCH = 200
#     NB_SPLITS = 4
#     mskf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, random_state = 0, shuffle = True)

#     oof_preds = []
#     oof_targets = []
#     # scores = []
#     scores_auc = []
#     best_score = 9999
#     for param in ParameterGrid(tabnet_params_2):
#         param['n_a'] = param['n_d']
#         print(param)
#         param_scores = []
#         for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train_step2, target_step2)):
#             print(b_,"FOLDS: ", r_, fold_nb + 1)
#             print(g_, '*' * 60, c_)

#             X_train, y_train = train_step2.values[train_idx, :], target_step2.values[train_idx, :]
#             X_val, y_val = train_step2.values[val_idx, :], target_step2.values[val_idx, :]
#             ### Model ###
#             model = TabNetRegressor(**param)

#             ### Fit ###
#             model.fit(
#                 X_train = X_train,
#                 y_train = y_train,
#                 eval_set = [(X_val, y_val)],
#                 eval_name = ["val"],
#                 eval_metric = ["logits_ll"],
#                 max_epochs = MAX_EPOCH,
#                 patience = 20,
#                 batch_size = 1024, 
#                 virtual_batch_size = 32,
#                 num_workers = 1,
#                 drop_last = False,
#                 loss_fn = BCEWithLogitsLoss())
#             print(y_, '-' * 60)

#             ### Predict on validation ###
#             preds_val = model.predict(X_val)
#             # Apply sigmoid to the predictions
#             preds = 1 / (1 + np.exp(-preds_val))
#             score = np.min(model.history["val_logits_ll"])

#             oof_preds.append(preds_val)
#             oof_targets.append(y_val)
#             param_scores.append(score)
#         current_score = np.mean(param_scores)
#         if current_score< best_score:
#             print('score was improved from {} to {}'.format(best_score, current_score))
#             best_param = param
#             chck.write(best_param, +'\n')
#             best_score = current_score
#             chck.write(best_score, +'\n')
#             ### Predict on test ###
 

## Train the TabNet with the best parameters

In [31]:
model = TabNetRegressor(**best_tabnet_params)
model.fit(
            X_train = train_step2.values,
            y_train = target_step2.values,
            max_epochs = MAX_EPOCH,
            patience = 20,
            batch_size = 1024, 
            virtual_batch_size = 32,
            num_workers = 1,
            drop_last = False,
            loss_fn = BCEWithLogitsLoss())


Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.14937 |  0:00:08s
epoch 10 | loss: 0.02022 |  0:01:35s
epoch 20 | loss: 0.01893 |  0:03:04s
epoch 30 | loss: 0.01753 |  0:04:31s
epoch 40 | loss: 0.01653 |  0:05:59s
epoch 50 | loss: 0.01581 |  0:07:28s
epoch 60 | loss: 0.01563 |  0:08:55s
epoch 70 | loss: 0.01547 |  0:10:24s
epoch 80 | loss: 0.01552 |  0:12:04s
epoch 90 | loss: 0.01542 |  0:13:43s


## get the final (scored) columns prediction on test set



In [32]:
preds_test = model.predict(X_test_step2)
preds_test = 1 / (1 + np.exp(-preds_test))
preds_test.shape

(3624, 206)

In [ ]:
# aucs = []
# for task_id in range(oof_preds_all.shape[1]):
#     aucs.append(roc_auc_score(y_true = oof_targets_all[:, task_id],
#                               y_score = oof_preds_all[:, task_id]
#                              ))
# print(f"{b_}Overall AUC: {r_}{np.mean(aucs)}")
# print(f"{b_}Average CV: {r_}{np.mean(scores)}")


#without augmentation and chain:
# Overall AUC: 0.7530900378764305
# Average CV: 0.016472221058818622

## Prepare submission file

In [33]:
all_feat = [col for col in df.columns if col not in ["sig_id"]]
# To obtain the same lenght of test_preds_all and submission
test = pd.read_csv("test_features.csv")
sig_id = test[test["cp_type"] != "ctl_vehicle"].sig_id.reset_index(drop = True)
tmp = pd.DataFrame(preds_test, columns = all_feat)
tmp["sig_id"] = sig_id

submission = pd.merge(test[["sig_id"]], tmp, on = "sig_id", how = "left")
submission.fillna(0, inplace = True)
submission.to_csv("submission.csv", index = None)
submission.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000715,0.000204,0.001482,0.019551,0.019379,0.005304,0.002348,0.002891,0.000176,...,0.000341,0.000333,0.004546,0.002004,0.002069,0.000486,0.001449,0.001377,0.001446,0.001757
1,id_001897cda,0.001066,0.000320,0.001595,0.005118,0.002441,0.001820,0.005874,0.002514,0.005877,...,0.000420,0.000255,0.006475,0.000104,0.001380,0.000547,0.009870,0.000963,0.017641,0.001017
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000663,0.000231,0.001302,0.017847,0.016440,0.005537,0.002343,0.003319,0.000197,...,0.000341,0.000348,0.004149,0.004139,0.003059,0.000488,0.002337,0.001531,0.000724,0.001887
4,id_0027f1083,0.001032,0.000287,0.001394,0.012617,0.011547,0.003768,0.002589,0.002745,0.000360,...,0.000375,0.000269,0.004460,0.001001,0.002316,0.000557,0.004362,0.001484,0.000692,0.001848
